following this video first to understand backprop... https://youtu.be/VMj-3S1tku0?si=eP7Ai1CO0BHh-yeS 

also note: to get this working on vscode, needed to install virtual environments T_T 

In [3]:
import math
import numpy as np 
import matplotlib.pyplot as plt

(at first the videos covers gradients, the "speed of change" at any point), then moving onto the structure of "micrograd"

In [9]:
class Value:
    # this is the constructor function in python classes 
    def __init__(self, data,): 
        self.data = data 
    
    # this defines the string REPRESENTATION of the object 
    def __repr__(self): 
        return f"Value(data={self.data})"
    
    # we need to enable adding as well of these Value objects
    def __add__(self, other):
        out = Value(self.data + other.data) # creates a new value object
        return out

    def __mul__(self, other):
        out = Value(self.data * other.data) 
        return out


In [10]:
a = Value(2.0)
b = Value(-3.0)
a + b # doesn't work until... implemented, now a.__add__(b) under the hood
a * b # doesn't work until implementing __mul__

Value(data=-1.0)